Import required packages and load card names.

In [58]:
import re

with(open('decklist.mtgo', 'r')) as decklist:
    card_names = decklist.readlines()
    card_names = [re.sub('[1-9]* ', '',  card.strip()) for card in card_names]

del decklist

Get card information

In [59]:
def get_card_info(cardjs):
    layout = cardjs['layout']
    oracle = []
    crop = ""
    if layout == 'split':
        for face in cardjs["card_faces"]:
            oracle.append(face['oracle_text'])
        crop = cardjs["image_uris"]["border_crop"]
    elif layout == 'transform':
        for face in cardjs["card_faces"]:
            oracle.append(face['oracle_text'])
        crop = cardjs["card_faces"][0]["image_uris"]["border_crop"]
    else:
        oracle.append(cardjs['oracle_text'])
        crop = cardjs["image_uris"]["border_crop"]
    name = re.sub("[^a-z1-9-]*", "", cardjs['name'].lower().replace(" ", "-"))
    if crop == "":
        print(json.dumps(cardjs,indent=4))
        exit
    return([name, crop, oracle])

In [60]:
import requests
import json
import time

api_url = "https://api.scryfall.com/cards"
cards = []
for card in card_names:
    card_resp = requests.get(f"{api_url}/named?exact={card}")
    if card_resp.status_code == 200:
        cardjs = json.loads(card_resp.text)
        cards.append(get_card_info(cardjs))
    elif card_resp.status_code == 429:
        print(f"Got 429, backing off.")
        exit
    else:
        print(f"Error getting {card}.")
        print(card_resp)
    time.sleep(0.05)

Error getting .
<Response [400]>


In [61]:
import os
import shutil

os.makedirs("img", exist_ok=True)
os.makedirs("pdf", exist_ok=True)
for card in cards:
    with(open(f"img/{card[0]}.jpg", "wb")) as imagefile:
        img_resp = requests.get(card[1], stream=True)
        if(img_resp.status_code == 429):
            print("Got 429, backing off")
            exit
        elif(img_resp.status_code == 200):
            shutil.copyfileobj(img_resp.raw, imagefile)
        else:
            print(f"Error downloading image for {card[0]}.")
        time.sleep(0.05)
    del imagefile

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet
ss = getSampleStyleSheet()['BodyText']
ss.fontSize = 10
x_offset = 5
y_offset = 5
x_count = 2
y_count = 5
width, height = A4
cellwidth = (width-2*x_offset) / x_count
cellheight = (height-2*y_offset) / y_count
cells = [(cellwidth*j+x_offset, cellheight*i+y_offset)  for i in range(0,y_count+1) for j in range(0,x_count+1)]

draw_cells = filter(lambda cell: cell[0]<(width-x_offset) and cell[1]<(height-y_offset), sorted(cells))
pdf = canvas.Canvas("pdf/cards.pdf", pagesize=A4)
pdf.grid([cell[0] for cell in cells], [cell[1] for cell in cells])

i = 0
for card in cards:
    text = "<br/>".join(card[2])
    i = i+1 
    cell = next(draw_cells)
    imgw, imgh = pdf.drawImage(f"img/{card[0]}.jpg", cell[0]+x_offset, cell[1]+y_offset, height=cellheight-10, anchor='w',preserveAspectRatio=True)
    width_offset = (imgw*((cellheight-10)/imgh)+2*x_offset)
    p = Paragraph(text, ss)
    w, h = p.wrapOn(pdf, cellwidth-width_offset-5, cellheight-y_offset)
    while(h>cellheight-2*y_offset):
        ss.fontSize = ss.fontSize - 1
        p = Paragraph(p.getPlainText(), ss)
        _, h = p.wrapOn(pdf, cellwidth-width_offset-5, cellheight-y_offset)
    p.drawOn(pdf, cell[0]+width_offset, cell[1]+(cellheight-h)-y_offset)
    ss.fontSize = 10
    if i == x_count*y_count:
        pdf.showPage()
        draw_cells = filter(lambda cell: cell[0]<(width-x_offset) and cell[1]<(height-y_offset), sorted(cells))
        pdf.grid([cell[0] for cell in cells], [cell[1] for cell in cells])
        i = 0
pdf.save()